# Run This Notebook

<h3><a href="https://hub.openveda.cloud/hub/user-redirect/git-pull?repo=https://github.com/NASA-IMPACT/veda-docs&urlpath=lab/tree/veda-docs/user-guide/notebooks/tutorials/mapping-slider.ipynb&branch=main">🚀 Launch in VEDA JupyterHub (requires access)</a></h3>

<h4> To obtain credentials to VEDA Hub, <a href = "https://docs.openveda.cloud/user-guide/scientific-computing/getting-access.html"> follow this link for more information.</a></h4>

<div class="alert alert-block" style="
     background-color: #f8d7da;
     color: #721c24;
     border-left: 4px solid #28a745;
  ">
Disclaimer: it is highly recommended to run a tutorial within NASA VEDA JupyterHub, which already includes functions for processing and visualizing data specific to VEDA stories. Running the tutorial outside of the VEDA JupyterHub may lead to errors, specifically related to EarthData authentication. Additionally, it is recommended to use the Pangeo workspace within the VEDA JupyterHub, since certain packages relevant to this tutorial are already installed. </div>

<h4> If you <strong>do not</strong> have a VEDA Jupyterhub Account you can launch this notebook on your local environment using MyBinder by clicking the icon below.</h4>
<br/>
<a href="https://binder.openveda.cloud/v2/gh/NASA-IMPACT/veda-docs/9c8cdbae92906fb7062b8a0c759dad90e223a4f9?urlpath=lab%2Ftree%2Fuser-guide%2Fnotebooks%2Ftutorials%2Fmapping-slider.ipynb">
<img src="https://binder.openveda.cloud/badge_logo.svg" alt="Binder" title="A cute binder" width="150"/> </a>

## Environment Setup

If running this notebook outside of the VEDA JupyterHub, install the following packages:

In [2]:
# Load libraries

import requests
import matplotlib.pyplot as plt
import plotutils as putils
from pystac_client import Client
import folium

In [3]:
# For retrieving data already catalogued in VEDA STAC
STAC_API_URL = "https://openveda.cloud/api/stac"
RASTER_API_URL = "https://openveda.cloud/api/raster"

# Open STAC client designed for interacting with SpatioTemporal Asset Catalog (STAC) APIs and Catalogs
client_STAC = Client.open(STAC_API_URL)

This example will show how to create an interactive mapping slider comparison experience similar to what is used in the VEDA environment. Using an example dataset from the VEDA STAC catalog, this notebook utilizes the Folium package to create this experience. 

## Processing steps:
1.) Choose collection ID from the STAC catalog and two dates for analysis<br />
2.) Retrieve collection information and items from VEDA STAC catalog<br />
3.) Retrieve item statistics and tiling information<br />
4.) Plot data<br />

## Choose variable and retrieve json from VEDA STAC catalogue -HD Black Marble Nighttime Lights

In [5]:
date_pre = "2023-03-15"  # Select date before tornado 

collection_id = "delta-disasters-hd-blackmarble-nightlights"

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────

results_pre = client_STAC.search(collections=[collection_id], datetime=date_pre)
items_pre = list(results_pre.items())
item_pre = items_pre[0]
collection_pre = item_pre.get_collection()

dashboard_render_pre = collection_pre.extra_fields["renders"]["dashboard"]
assets_pre = dashboard_render_pre["assets"][0]
((vmin_pre, vmax_pre),) = dashboard_render_pre["rescale"]
bidx_pre = dashboard_render_pre.get("bidx", [3, 2, 1])

# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────

response_pre = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item_pre.id}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets_pre}"
    f"&bidx={bidx_pre[0]}&bidx={bidx_pre[1]}&bidx={bidx_pre[2]}"
    f"&rescale={vmin_pre},{vmax_pre}"
    f"&resampling=bilinear"
)
response_pre.raise_for_status()
tiles_pre = response_pre.json()
print(tiles_pre)

collection_pre

{'tilejson': '2.2.0', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://openveda.cloud/api/raster/collections/delta-disasters-hd-blackmarble-nightlights/items/HD_Blackmarble_Nightlights_2023-03-15/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=cog_default&bidx=1&bidx=2&bidx=3&rescale=0%2C255&resampling=bilinear'], 'minzoom': 0, 'maxzoom': 24, 'bounds': [-90.95641041935056, 32.83238037641129, -90.78794047564391, 32.96655224164907], 'center': [-90.87217544749723, 32.899466309030174, 0]}


<CollectionClient id=delta-disasters-hd-blackmarble-nightlights>

In [6]:
date_post = "2023-03-25"  # Select date after tornado 

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────

results_post = client_STAC.search(collections=[collection_id], datetime=date_post)

items_post = list(results_post.items())
assert len(items_post) != 0, "No items found"
item_post = items_post[0]
collection_post = item_post.get_collection()

# grab the dashboard render block
dashboard_render_post = collection_post.extra_fields["renders"]["dashboard"]

assets_post = dashboard_render_post["assets"][0]
((vmin_post, vmax_post),) = dashboard_render_post["rescale"]

# Special handling for RGB bands - post tornado uses different band order (1,2,3)
bidx_post = dashboard_render_post.get("bidx", [1, 2, 3])  # RGB band order for post


# ── VEDA Tile Request for Post-Tornado ─────────────────────────────────────────────────────────────────────

response_post = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item_post.id}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets_post}"
    f"&bidx={bidx_post[0]}&bidx={bidx_post[1]}&bidx={bidx_post[2]}"  # RGB bands (1,2,3 for post)
    f"&rescale={vmin_post},{vmax_post}"
    f"&resampling=bilinear"
)

response_post.raise_for_status()

tiles_post = response_post.json()
print(tiles_post)

collection_post


{'tilejson': '2.2.0', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://openveda.cloud/api/raster/collections/delta-disasters-hd-blackmarble-nightlights/items/HD_Blackmarble_Nightlights_2023-03-25/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=cog_default&bidx=1&bidx=2&bidx=3&rescale=0%2C255&resampling=bilinear'], 'minzoom': 0, 'maxzoom': 24, 'bounds': [-90.95641041935056, 32.83238037641129, -90.78794047564391, 32.96655224164907], 'center': [-90.87217544749723, 32.899466309030174, 0]}


<CollectionClient id=delta-disasters-hd-blackmarble-nightlights>

In [7]:
# First we will present the different basemaps that we have access to underlay beneath our tile requests
# For the first map, we will utilize the 'esri-satellite-labels' map layer
putils.get_available_basemaps()

{'openstreetmap': 'OpenStreetMap standard tiles',
 'cartodb-positron': 'Light gray CartoDB basemap (subtle, good for data visualization)',
 'cartodb-dark': 'Dark CartoDB basemap (good for bright data)',
 'esri-satellite': 'ESRI satellite imagery without labels',
 'esri-satellite-labels': 'ESRI satellite imagery with place labels overlay',
 None: 'No basemap (transparent background)'}

In [11]:
# Create side-by-side comparison of the effects of the March 24, 2023 tornado using the create_side_by_side_map function
m_comparison = putils.plot_folium_SidebySide_layer_from_VEDA_STAC(
    tiles_url_left=tiles_pre["tiles"][0],  # Pre-tornado (May 20)
    tiles_url_right=tiles_post["tiles"][0],  # Post-tornado (May 22)
    center_coords=[32.9069, -90.8785],  # Rolling Fork, MS
    zoom_level=13.5,
    title="NASA's HD Black Marble Nighttime Lights: Rolling Fork, MS EF-4 Tornado",
    label_left="← Pre-Tornado (March 15)",
    label_right="Post-Tornado (March 25) →",
    layer_name_left="Pre-Tornado",
    layer_name_right="Post-Tornado",
    opacity=1.0,  # Full opacity for satellite imagery
    basemap_style='cartodb-positron',  # Light basemap
    height="800px",
    width="100%"
)

# Display the comparison map
m_comparison